In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib2
% matplotlib inline

In [135]:
date = "150516" ## LATER: update to pull most recent week or take an argument
url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_" + date + ".txt"
data = urllib2.urlopen(url)
df = pd.read_csv(data)

In [136]:
df.head()

## it works!!

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/09/2015,00:00:00,REGULAR,5128029,1735976
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/09/2015,04:00:00,REGULAR,5128029,1735976
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/09/2015,08:00:00,REGULAR,5128051,1735997
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/09/2015,12:00:00,REGULAR,5128186,1736096
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/09/2015,16:00:00,REGULAR,5128457,1736177


In [137]:
## check for irregular rows

irreg = sum(df.DESC != "REGULAR")

recoverd = sum(df.DESC == "RECOVR AUD")

## verify that all irregular rows are "recovered audits"
assert(irreg == recoverd)

In [138]:
## filter out PATH trains (DIVISION = "PTH")
lenbefore = len(df)
df2 = df[df.DIVISION != "PTH"]
lenafter = len(df2)

assert(lenbefore > lenafter)
assert(sum(df2.DIVISION == "PTH") == 0)
df = df2

In [139]:
## aggregate by station (we don't care about individual devices)
##  while preserving date/time data

## GROUP_BY station, linename, date, time AGGREGATE entries, exits BY sum
aggdf = df.groupby(["STATION","LINENAME","DATE","TIME"]).sum()
aggdf.head(100)
## some stations share a name, but no stations on the same line share a name, thus use of both STATION and LINENAME
##  (some stations also have multiple UNITs)

ENTRIES  \
STATION LINENAME DATE       TIME                  
1 AVE   L        05/09/2015 00:00:00  464522607   
                            04:00:00  464522608   
                            08:00:00  464522611   
                            12:00:00  464522612   
                            16:00:00  464522614   
                            20:00:00  464522615   
                 05/10/2015 00:00:00  464522619   
                            04:00:00  464522619   
                            08:00:00  464522623   
                            12:00:00  464522623   
                            16:00:00  464522624   
                            20:00:00  464522624   
                 05/11/2015 00:00:00  464522631   
                            04:00:00  464522635   
                            08:00:00  464524518   
                            12:00:00  464529678   
                            16:00:00  464534546   
                            20:00:00  464541265   
                 05/12/2015 00:00:00  464544565   
                            04:00:00  464544577   
                            08:00:00  464546511   
                            12:00:00  464551934   
                            16:00:00  464556795   
                            20:00:00  464563931   
                 05/13/2015 00:00:00  464567690   
                            04:00:00  464567692   
                            08:00:00  464569701   
                            12:00:00  464575360   
                            16:00:00  464580296   
                            20:00:00  464587699   
...                                         ...   
103 ST  1        05/13/2015 17:00:00   60907958   
                            21:00:00   60910946   
                 05/14/2015 01:00:00   60911844   
                            05:00:00   60911931   
                            09:00:00   60915529   
                            13:00:00   60919057   
                            17:00:00   60922092   
                            21:00:00   60925279   
                 05/15/2015 01:00:00   60926379   
                            05:00:00   60926484   
                            09:00:00   60929703   
                            13:00:00   60933173   
                            17:00:00   60936202   
                            21:00:00   60939498   
        6        05/09/2015 01:00:00   36442007   
                            05:00:00   36442136   
                            09:00:00   36442835   
                            13:00:00   36444076   
                            17:00:00   36445634   
                            21:00:00   36446755   
                 05/10/2015 01:00:00   36447317   
                            05:00:00   36447424   
                            09:00:00   36447893   
                            13:00:00   36448954   
                            17:00:00   36450131   
                            21:00:00   36451033   
                 05/11/2015 01:00:00   36451485   
                            05:00:00   36451567   
                            09:00:00   36453550   
                            13:00:00   36455274   

                                      EXITS                                                                 
STATION LINENAME DATE       TIME                                                                            
1 AVE   L        05/09/2015 00:00:00                                          478747341                     
                            04:00:00                                          478747389                     
                            08:00:00                                          478747458                     
                            12:00:00                                          478747468                     
                            16:00:00                                          478747471                     
                            20:00:00                           

In [140]:
## some rows are missing -- 103rd St 1 line 8/6/2015 at 9:00am reading missing.
##  how do we identify more such rows?
##    after subtracting prv row value, these rows should be the only ones with negative values
##  how do we fix them?
##    a. remove row entirely (won't someone PLEASE think of the DATA!!)
##    b. interpolate (split the diff btwn next and previous rows) (ideal, but futzier)
##    c. subtract cumulative totals before agg'ing (missing entries will accrue to next row) (easier to implement)

## subtract previous row values for all entries and exits to get total during period
agg = df.groupby(["UNIT", "SCP", "STATION","LINENAME"])
obj = pd.DataFrame()
for name, group in agg:
    #print name
    group = group.reset_index(drop=True)
    #print group.values[0]
    nc_ent = [0]
    nc_ext = [0]
    for i in range(1,len(group)):
        nc_ent.append(group.values[i][9]-group.values[i-1][9])
        nc_ext.append(group.values[i][10]-group.values[i-1][10])
    out = pd.concat([group,
                     pd.Series(nc_ent,name="NC_ENTRIES"),
                     pd.Series(nc_ext,name="NC_EXITS")], axis=1)
    obj = pd.concat([obj,out],ignore_index=True)

obj.describe()

,ENTRIES,EXITS,NC_ENTRIES,NC_EXITS
count,1.822250e+05,1.822250e+05,1.822250e+05,1.822250e+05
mean,3.082787e+07,2.480127e+07,1.345344e+04,1.558693e+04
std,1.830777e+08,1.637886e+08,2.901951e+06,2.953083e+06
min,0.000000e+00,0.000000e+00,-3.104217e+06,-6.188433e+06
25%,6.454080e+05,3.444830e+05,1.000000e+01,8.000000e+00
50%,2.587049e+06,1.574003e+06,8.200000e+01,5.600000e+01
75%,6.240699e+06,4.553139e+06,2.680000e+02,1.790000e+02
max,2.114257e+09,2.087387e+09,1.036157e+09,7.352539e+08


In [141]:
obj[obj.NC_ENTRIES < 0]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,NC_ENTRIES,NC_EXITS
2201,J034,R007,00-00-02,104 ST,JZ,BMT,05/09/2015,04:00:00,REGULAR,2026256283,622292498,-8,-11
2202,J034,R007,00-00-02,104 ST,JZ,BMT,05/09/2015,08:00:00,REGULAR,2026256240,622292490,-43,-8
2203,J034,R007,00-00-02,104 ST,JZ,BMT,05/09/2015,12:00:00,REGULAR,2026256190,622292464,-50,-26
2204,J034,R007,00-00-02,104 ST,JZ,BMT,05/09/2015,16:00:00,REGULAR,2026256128,622292427,-62,-37
2205,J034,R007,00-00-02,104 ST,JZ,BMT,05/09/2015,20:00:00,REGULAR,2026256106,622292365,-22,-62
2206,J034,R007,00-00-02,104 ST,JZ,BMT,05/10/2015,00:00:00,REGULAR,2026256086,622292326,-20,-39
2207,J034,R007,00-00-02,104 ST,JZ,BMT,05/10/2015,04:00:00,REGULAR,2026256077,622292317,-9,-9
2208,J034,R007,00-00-02,104 ST,JZ,BMT,05/10/2015,08:00:00,REGULAR,2026256049,622292310,-28,-7
2209,J034,R007,00-00-02,104 ST,JZ,BMT,05/10/2015,12:00:00,REGULAR,2026256004,622292292,-45,-18
2210,J034,R007,00-00-02,104 ST,JZ,BMT,05/10/2015,16:00:00,REGULAR,2026255952,622292263,-52,-29


In [7]:
df.describe()

,ENTRIES,EXITS
count,1.912290e+05,1.912290e+05
mean,3.098024e+07,2.549711e+07
std,1.827583e+08,1.663129e+08
min,0.000000e+00,0.000000e+00
25%,5.017000e+05,2.746610e+05
50%,2.302886e+06,1.388752e+06
75%,6.064040e+06,4.406976e+06
max,2.118720e+09,2.087387e+09
